In [5]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import matplotlib.pyplot as plt
import random
from numpy import save

2021-12-15 02:44:47.853819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 02:44:47.853991: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
df=pd.read_csv('full_df.csv')

In [7]:
df3= df[(df['labels']== "['D']")&(df.D==1)]

In [8]:
df2= df[(df['labels']!= "['D']")& (df.D==0)]

In [9]:
len(df2)

4269

In [10]:
len(df3)

1608

In [11]:
NoDiab=df2.filename

In [12]:
NoDiabPrueba=NoDiab[-30:]

In [13]:
NoDiabTesteo=NoDiab[:-30]

In [36]:
len(NoDiabPrueba)

30

In [22]:
Diab=df3.filename

In [23]:
DiabPrueba=Diab[-30:]

In [24]:
DiabTesteo=Diab[:-30]

In [25]:
len(DiabPrueba)

30

In [86]:
def create_dataset(image_category,label):
    dataset_dir = "Datos/preprocessed_2"
    image_size=250
    labels = []
    dataset = []
    for img in image_category:
        image_path = os.path.join(dataset_dir,img)
        try:
            img = cv2.imread(image_path,0)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            img = img[..., np.newaxis]
            
        except:
            continue
        
        dataset.append([np.array(img),np.array(label)])
    random.shuffle(dataset)
    return dataset

In [165]:
def augment(category,label,mult):
    x=[]
    for img in category:
        try:
            img = cv2.imread(f"Datos/preprocessed_2/{img}",0)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            img = img[..., np.newaxis]
        except:
            continue
        train_datagen = ImageDataGenerator(
                    rotation_range=20,
                    zoom_range=0.2,
                    brightness_range=(0.4, 1.2),
                    height_shift_range=0.2,
                    )
        img = img.reshape((1,) + img.shape)
        
        it = train_datagen.flow(x=img, batch_size=1)
        for i in range(mult):
            batch = it.next()
            aug_img = batch[0]
            aug_img=aug_img.reshape(-1)
            x.append([np.array(img),np.array(label)])
            
       
    for i in range(len(x)):
        x[i][0]=x[i][0].reshape(x[i][0].shape[1:])
    return x

In [3]:
def create_dataset2(image_category,label):
    dataset_dir = "Datos/preprocessed_2"
    image_size=250
    labels = []
    dataset = []
    for img in image_category:
        image_path = os.path.join(dataset_dir,img)
        try:
            img = cv2.imread(image_path,cv2.IMREAD_COLOR)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            
            
        except:
            continue
        
        dataset.append([np.array(img),np.array(label)])
    return dataset

In [37]:
def augment2(category,label,mult):
    x=[]
    for img in category:
        try:
            img = cv2.imread(f"Datos/preprocessed_2/{img}",cv2.IMREAD_COLOR)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
        except:
            continue
        train_datagen = ImageDataGenerator(
                    rotation_range=20,
                    zoom_range=0.2,
                    brightness_range=(0.4, 1.2),
                    height_shift_range=0.2,
                    )
        img = img.reshape((1,) + img.shape)
        
        it = train_datagen.flow(x=img, batch_size=1)
        for i in range(mult):
            batch = it.next()
            aug_img = batch[0]
            aug_img=aug_img.reshape(-1)
            x.append([np.array(img),np.array(label)])
            
       
    for i in range(len(x)):
        x[i][0]=x[i][0].reshape(x[i][0].shape[1:])
    return x

In [18]:
Diab_prueba=create_dataset2(DiabPrueba,1)

In [21]:
Diab_prueba[0][0].shape

(250, 250, 3)

In [35]:
NoDiab_prueba=create_dataset(NoDiabPrueba,0)

In [78]:
DiabetesTotalPrueba=Diab_prueba+NoDiab_prueba

In [122]:
len(DiabetesTotalPrueba)

60

In [123]:
x_test = np.array([i[0]for i in DiabetesTotalPrueba])
y_test = np.array([i[1] for i in DiabetesTotalPrueba])

In [124]:
len(x_test)

60

In [38]:
Diab_Aug=augment2(DiabTesteo,1,1)

In [167]:
len(Diab_Aug)

1578

In [111]:
#NoDiab_Aug=augment(NoDiabTesteo,0,1)

In [26]:
Nodiabetes=create_dataset2(NoDiabTesteo,0)

In [39]:
len(Nodiabetes)

4239

In [28]:
Diabetes=create_dataset2(DiabTesteo,1)

In [40]:
len(Diabetes)

1578

In [41]:
Totaldetotal=Diabetes+Nodiabetes+Diab_Aug

In [42]:
len(Totaldetotal)

7395

In [43]:
x = np.array([i[0]for i in Totaldetotal])
y = np.array([i[1] for i in Totaldetotal])

In [44]:
x.shape

(7395, 250, 250, 3)

In [45]:
save('x',x)

In [46]:
save('y',y)

In [38]:
import h5py
import cv2
from keras.models import load_model
import numpy as np

In [175]:
model = load_model("my_model_diabetes7.h5")

In [176]:
y_pred = (model.predict(x_test) > 0.5).astype("int32")

In [2]:
len(y_pred)

NameError: name 'y_pred' is not defined

In [179]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score

In [180]:
precision_score(
y_test,
y_pred)

0.7222222222222222

In [181]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.60      0.83      0.69        30
           1       0.72      0.43      0.54        30

    accuracy                           0.63        60
   macro avg       0.66      0.63      0.62        60
weighted avg       0.66      0.63      0.62        60

